In [ ]:
#|default_exp toolloop

# Tool loop

In [ ]:
import os
# os.environ['ANTHROPIC_LOG'] = 'debug'

In [ ]:
#| export
from claudette.core import *
from fastcore.utils import *
from fastcore.meta import delegates

from anthropic.types import TextBlock, Message, ToolUseBlock

In [ ]:
model = models[-1]

Anthropic provides an [interesting example](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/customer_service_agent.ipynb) of using tools to mock up a hypothetical ordering system. We're going to take it a step further, and show how we can dramatically simplify the process, whilst completing more complex tasks.

We'll start by defining the same mock customer/order data as in Anthropic's example, plus create a entity relationship between customers and orders:

In [ ]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

We can now define the same functions from the original example -- but note that we don't need to manually create the large JSON schema, since Claudette handles all that for us automatically from the functions directly. We'll add some extra functionality to update order details when cancelling too.

In [ ]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

We're now ready to start our chat.

In [ ]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(model, tools=tools)

We'll start with the same request as Anthropic showed:

In [ ]:
r = chat('Can you tell me the email address for customer C1?')
print(r.stop_reason)
r.content

- Retrieving customer C1
tool_use


[TextBlock(citations=None, text="I'll retrieve the customer information for customer ID C1.", type='text'),
 ToolUseBlock(id='toolu_01BBozt1ZDnbBHMJ8bxVKLoE', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Claude asks us to use a tool. Claudette handles that automatically by just calling it again:

In [ ]:
r = chat()
contents(r)

'The email address for customer C1 (John Doe) is john@example.com.\n\nIs there anything else I can help you with?'

Let's consider a more complex case than in the original example -- what happens if a customer wants to cancel all of their orders?

In [ ]:
chat = Chat(model, tools=tools)
r = chat('Please cancel all orders for customer C1 for me.')
print(r.stop_reason)
r.content

- Retrieving customer C1
tool_use


[TextBlock(citations=None, text="I'll help you cancel all orders for customer C1. To do this, I'll first retrieve the customer's information to get their order details, and then cancel each order.", type='text'),
 ToolUseBlock(id='toolu_01DawhHveHK4cWLV7XwSXkMw', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

This is the start of a multi-stage tool use process. Doing it manually step by step is inconvenient, so let's write a function to handle this for us:

In [ ]:
#| exports
@patch
@delegates(Chat.__call__)
def toolloop(self:Chat,
             pr, # Prompt to pass to Claude
             max_steps=10, # Maximum number of tool requests to loop through
             trace_func:Optional[callable]=None, # Function to trace tool use steps (e.g `print`)
             cont_func:Optional[callable]=noop, # Function that stops loop if returns False
             **kwargs):
    "Add prompt `pr` to dialog and get a response from Claude, automatically following up with `tool_use` messages"
    init_n = n_msgs = len(self.h)
    r = self(pr, **kwargs)
    for i in range(max_steps):
        if r.stop_reason!='tool_use': break
        if trace_func: trace_func(self.h[n_msgs:]); n_msgs = len(self.h)
        r = self(**kwargs)
        if not (cont_func or noop)(self.h[-2]): break
    if trace_func: trace_func(self.h[n_msgs:])
    r.steps = self.h[init_n:]
    return r

We'll start by re-running our previous request - we shouldn't have to manually pass back the `tool_use` message any more:

In [ ]:
chat = Chat(model, tools=tools)
r = chat.toolloop('Can you tell me the email address for customer C1?')
r

- Retrieving customer C1


The email address for customer C1 (John Doe) is john@example.com.

Is there anything else I can help you with?

<details>

- id: `msg_01LPTK7ubKAQnbFRrbVQ9Gwq`
- content: `[{'citations': None, 'text': 'The email address for customer C1 (John Doe) is john@example.com.\n\nIs there anything else I can help you with?', 'type': 'text'}]`
- model: `claude-3-5-haiku-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 732, 'output_tokens': 35}`
- steps: `[{'role': 'user', 'content': 'Can you tell me the email address for customer C1?'}, {'role': 'assistant', 'content': [{'citations': None, 'text': "I'll retrieve the customer information for customer ID C1.", 'type': 'text'}, {'id': 'toolu_01SSHEbn69FBxoRdNhpDwxz2', 'input': {'customer_id': 'C1'}, 'name': 'get_customer_info', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01SSHEbn69FBxoRdNhpDwxz2', 'content': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Processing'}]}"}]}]`

</details>

Let's see if it can handle the multi-stage process now -- we'll add `trace_func=print` to see each stage of the process:

In [ ]:
chat = Chat(model, tools=tools)
r = chat.toolloop('Please cancel all orders for customer C1 for me.', trace_func=print)
r

- Retrieving customer C1
[{'role': 'user', 'content': 'Please cancel all orders for customer C1 for me.'}, {'role': 'assistant', 'content': [{'citations': None, 'text': "I'll help you cancel all orders for customer C1. To do this, I'll first retrieve the customer's information to get their order details, and then cancel each order.", 'type': 'text'}, {'id': 'toolu_01AQPh1S4AcHqdZCw2S3zX5j', 'input': {'customer_id': 'C1'}, 'name': 'get_customer_info', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01AQPh1S4AcHqdZCw2S3zX5j', 'content': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Processing'}]}"}]}]
- Cancelling order O1
[{'role': 'assistant', 'content': [{'citations': None, 'text': "I see that customer C1 (John Doe) has two orders:

I've successfully cancelled both orders (O1 and O2) for customer C1. Both cancellations returned 'True', which indicates they were processed successfully. Is there anything else I can help you with?

<details>

- id: `msg_01NWfWvATr2vVwkqpk5qCTJj`
- content: `[{'citations': None, 'text': "I've successfully cancelled both orders (O1 and O2) for customer C1. Both cancellations returned 'True', which indicates they were processed successfully. Is there anything else I can help you with?", 'type': 'text'}]`
- model: `claude-3-5-haiku-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 926, 'output_tokens': 49}`
- steps: `[{'role': 'user', 'content': 'Please cancel all orders for customer C1 for me.'}, {'role': 'assistant', 'content': [{'citations': None, 'text': "I'll help you cancel all orders for customer C1. To do this, I'll first retrieve the customer's information to get their order details, and then cancel each order.", 'type': 'text'}, {'id': 'toolu_01AQPh1S4AcHqdZCw2S3zX5j', 'input': {'customer_id': 'C1'}, 'name': 'get_customer_info', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01AQPh1S4AcHqdZCw2S3zX5j', 'content': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Processing'}]}"}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': "I see that customer C1 (John Doe) has two orders: O1 and O2. I'll proceed to cancel both orders:", 'type': 'text'}, {'id': 'toolu_01Cn3E2L21LMsBq5hdwLZYqR', 'input': {'order_id': 'O1'}, 'name': 'cancel_order', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01Cn3E2L21LMsBq5hdwLZYqR', 'content': 'True'}]}, {'role': 'assistant', 'content': [{'id': 'toolu_01U1YjpJRqP6w79WrGWW8RJT', 'input': {'order_id': 'O2'}, 'name': 'cancel_order', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01U1YjpJRqP6w79WrGWW8RJT', 'content': 'True'}]}]`

</details>

OK Claude thinks the orders were cancelled -- let's check one:

In [ ]:
chat.toolloop('What is the status of order O2?')

- Retrieving order O2


The status of order O2 is now "Cancelled". This reflects the cancellation we just performed in the previous interaction. The order was for 1 Gadget B priced at &#36;49.99, and it has been successfully cancelled.

<details>

- id: `msg_01GgqDYyJaJ6QLbeQpNf2RMy`
- content: `[{'citations': None, 'text': 'The status of order O2 is now "Cancelled". This reflects the cancellation we just performed in the previous interaction. The order was for 1 Gadget B priced at $49.99, and it has been successfully cancelled.', 'type': 'text'}]`
- model: `claude-3-5-haiku-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1117, 'output_tokens': 56}`
- steps: `[{'role': 'user', 'content': 'What is the status of order O2?'}, {'role': 'assistant', 'content': [{'citations': None, 'text': "I'll retrieve the details of order O2 to check its current status:", 'type': 'text'}, {'id': 'toolu_017hBY8CizQGRqan7wsTQrFg', 'input': {'order_id': 'O2'}, 'name': 'get_order_details', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_017hBY8CizQGRqan7wsTQrFg', 'content': "{'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Cancelled'}"}]}]`

</details>

## Async tool loop

In [ ]:
#| export
from claudette.asink import AsyncChat

In [ ]:
#| exports
@patch
@delegates(AsyncChat.__call__)
async def toolloop(self:AsyncChat,
             pr, # Prompt to pass to Claude
             max_steps=10, # Maximum number of tool requests to loop through
             trace_func:Optional[callable]=None, # Function to trace tool use steps (e.g `print`)
             cont_func:Optional[callable]=noop, # Function that stops loop if returns False
             **kwargs):
    "Add prompt `pr` to dialog and get a response from Claude, automatically following up with `tool_use` messages"
    init_n = n_msgs = len(self.h)
    r = await self(pr, **kwargs)
    for i in range(max_steps):
        if r.stop_reason!='tool_use': break
        if trace_func: trace_func(self.h[n_msgs:]); n_msgs = len(self.h)
        r = await self(**kwargs)
        if not (cont_func or noop)(self.h[-2]): break
    if trace_func: trace_func(self.h[n_msgs:])
    r.steps = self.h[init_n+1:]
    return r

In [ ]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

In [ ]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = AsyncChat(model, tools=tools)
r = await chat.toolloop('Can you tell me the email address for customer C1?')
r

- Retrieving customer C1


The email address for customer C1 (John Doe) is john@example.com.

Is there anything else I can help you with?

<details>

- id: `msg_01S2RzwEK7BrD9wHcz22aSBZ`
- content: `[{'citations': None, 'text': 'The email address for customer C1 (John Doe) is john@example.com.\n\nIs there anything else I can help you with?', 'type': 'text'}]`
- model: `claude-3-5-haiku-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 740, 'output_tokens': 35}`
- steps: `[{'role': 'assistant', 'content': [{'citations': None, 'text': "I'll retrieve the customer information for customer C1 using the get_customer_info function.", 'type': 'text'}, {'id': 'toolu_012wRs4jo7deZWDbzHYD3Eqx', 'input': {'customer_id': 'C1'}, 'name': 'get_customer_info', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_012wRs4jo7deZWDbzHYD3Eqx', 'content': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Processing'}]}"}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': 'The email address for customer C1 (John Doe) is john@example.com.\n\nIs there anything else I can help you with?', 'type': 'text'}]}]`

</details>

Let's see if it can handle the multi-stage process now -- we'll add `trace_func=print` to see each stage of the process:

In [ ]:
chat = AsyncChat(model, tools=tools)
r = await chat.toolloop('Please cancel all orders for customer C1 for me.', trace_func=print)
r

- Retrieving customer C1
[{'role': 'user', 'content': 'Please cancel all orders for customer C1 for me.'}, {'role': 'assistant', 'content': [TextBlock(citations=None, text="I'll help you cancel all orders for customer C1. To do this, I'll first retrieve the customer's information to get their order details, and then cancel each order.", type='text'), ToolUseBlock(id='toolu_01NS8YaQDSZtJ5qCfLXrHM2p', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01NS8YaQDSZtJ5qCfLXrHM2p', 'content': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Processing'}]}"}]}]
- Cancelling order O1
[{'role': 'assistant', 'content': [TextBlock(citations=None, text="I see that customer C1 (John Doe) has two orde

I've successfully cancelled both orders (O1 and O2) for customer C1. Both cancellations returned 'True', which indicates they were processed successfully. Is there anything else I can help you with?

<details>

- id: `msg_018edqpNzD9bVfdvGQbPfChd`
- content: `[{'citations': None, 'text': "I've successfully cancelled both orders (O1 and O2) for customer C1. Both cancellations returned 'True', which indicates they were processed successfully. Is there anything else I can help you with?", 'type': 'text'}]`
- model: `claude-3-5-haiku-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 926, 'output_tokens': 49}`
- steps: `[{'role': 'user', 'content': 'Please cancel all orders for customer C1 for me.'}, {'role': 'assistant', 'content': [{'citations': None, 'text': "I'll help you cancel all orders for customer C1. To do this, I'll first retrieve the customer's information to get their order details, and then cancel each order.", 'type': 'text'}, {'id': 'toolu_01NS8YaQDSZtJ5qCfLXrHM2p', 'input': {'customer_id': 'C1'}, 'name': 'get_customer_info', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01NS8YaQDSZtJ5qCfLXrHM2p', 'content': "{'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Processing'}]}"}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': "I see that customer C1 (John Doe) has two orders: O1 and O2. I'll proceed to cancel both orders:", 'type': 'text'}, {'id': 'toolu_019GLnGm4WVLS62Mu38nCn88', 'input': {'order_id': 'O1'}, 'name': 'cancel_order', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_019GLnGm4WVLS62Mu38nCn88', 'content': 'True'}]}, {'role': 'assistant', 'content': [{'id': 'toolu_018ExELaCWKGVBXSpQ2pwyCd', 'input': {'order_id': 'O2'}, 'name': 'cancel_order', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_018ExELaCWKGVBXSpQ2pwyCd', 'content': 'True'}]}]`

</details>

In [ ]:
await chat.toolloop('What is the status of order O2?')

- Retrieving order O2


The status of order O2 is now "Cancelled". This reflects the cancellation we just performed in the previous interaction. The order was for 1 Gadget B priced at &#36;49.99, and it has been successfully cancelled.

<details>

- id: `msg_014ZnuWfK8wJeqHG6wkiTe9t`
- content: `[{'citations': None, 'text': 'The status of order O2 is now "Cancelled". This reflects the cancellation we just performed in the previous interaction. The order was for 1 Gadget B priced at $49.99, and it has been successfully cancelled.', 'type': 'text'}]`
- model: `claude-3-5-haiku-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1117, 'output_tokens': 56}`
- steps: `[{'role': 'user', 'content': 'What is the status of order O2?'}, {'role': 'assistant', 'content': [{'citations': None, 'text': "I'll retrieve the details of order O2 to check its current status:", 'type': 'text'}, {'id': 'toolu_01SBgGUT9KDRsNTL7RwW7Pc1', 'input': {'order_id': 'O2'}, 'name': 'get_order_details', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01SBgGUT9KDRsNTL7RwW7Pc1', 'content': "{'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Cancelled'}"}]}]`

</details>

## Code interpreter

Here is an example of using `toolloop` to implement a simple code interpreter with additional tools.

In [ ]:
from toolslm.shell import get_shell
from fastcore.meta import delegates
import traceback

In [ ]:
@delegates()
class CodeChat(Chat):
    imps = 'os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses'
    def __init__(self, model: Optional[str] = None, ask:bool=True, **kwargs):
        super().__init__(model=model, **kwargs)
        self.ask = ask
        self.tools.append(self.run_cell)
        self.shell = get_shell()
        self.shell.run_cell('import '+self.imps)

We have one additional parameter to creating a `CodeChat` beyond what we pass to `Chat`, which is `ask` -- if that's `True`, we'll prompt the user before running code.

In [ ]:
@patch
def run_cell(
    self:CodeChat,
    code:str,   # Code to execute in persistent IPython session
)->str:
    """Asks user for permission, and if provided, executes python `code` using persistent IPython session.
    Returns: Result of expression on last line (if exists); '#DECLINED#' if user declines request to execute"""
    confirm = f'Press Enter to execute, or enter "n" to skip?\n```\n{code}\n```\n'
    if self.ask and input(confirm): return '#DECLINED#'
    try: res = self.shell.run_cell(code)
    except Exception as e: return traceback.format_exc()
    return res.stdout if res.result is None else res.result

We just pass along requests to run code to the shell's implementation. Claude often prints results instead of just using the last expression, so we capture stdout in those cases.

In [ ]:
sp = f'''You are a knowledgable assistant. Do not use tools unless needed.
Don't do complex calculations yourself -- use code for them.
The following modules are pre-imported for `run_cell` automatically:

{CodeChat.imps}

Never mention what tools you are using. Note that `run_cell` interpreter state is *persistent* across calls.

If a tool returns `#DECLINED#` report to the user that the attempt was declined and no further progress can be made.
In that case, do *not* attempt to run any further code -- stop execution *IMMEDIATELY* and tell the user it was declined.'''

In [ ]:
def get_user()->str:
    "Get the username of the user running this session"
    print("Looking up username")
    return 'Jeremy'

In order to test out multi-stage tool use, we create a mock function that Claude can call to get the current username.

In [ ]:
model = models[1]
chat = CodeChat(model, tools=[get_user], sp=sp, ask=True, temp=0.3)

Providing a callable to toolloop's `trace_func` lets us print out information during the loop:

In [ ]:
def _show_cts(h):
    for r in h:
        for o in r.get('content'):
            if hasattr(o,'text'): print(o.text)
            nm = getattr(o, 'name', None)
            if nm=='run_cell': print(o.input['code'])
            elif nm: print(f'{o.name}({o.input})')

...and toolloop's `cont_func` callable let's us provide a function which, if it returns `False`, stops the loop:

In [ ]:
def _cont_decline(c): return nested_idx(c, 'content', 'content') != '#DECLINED#'

Now we can try our code interpreter. We start by asking for a function to be created, which we'll use in the next prompt to test that the interpreter is persistent.

In [ ]:
pr = '''Create a 1-line function `checksum` for a string `s`,
that multiplies together the ascii values of each character in `s` using `reduce`.'''
chat.toolloop(pr, temp=0.2, trace_func=_show_cts, cont_func=_cont_decline)

Press Enter to execute, or enter "n" to skip?
```
from functools import reduce

checksum = lambda s: reduce(lambda x, y: x * y, [ord(c) for c in s], 1)

# Test it with a simple example
test_string = "hello"
print(f"Checksum of '{test_string}': {checksum(test_string)}")
```

I'll create a 1-line function called `checksum` that multiplies together the ASCII values of each character in a string using `reduce`.
from functools import reduce

checksum = lambda s: reduce(lambda x, y: x * y, [ord(c) for c in s], 1)

# Test it with a simple example
test_string = "hello"
print(f"Checksum of '{test_string}': {checksum(test_string)}")
I've created a 1-line `checksum` function that:
1. Takes a string `s` as input
2. Converts each character to its ASCII value using `ord()`
3. Uses `reduce()` from the `functools` module to multiply all these values together
4. Starts with an initial value of 1 (the identity element for multiplication)

The function is defined as:
```python
checksum = lambda s: reduce

I've created a 1-line `checksum` function that:
1. Takes a string `s` as input
2. Converts each character to its ASCII value using `ord()`
3. Uses `reduce()` from the `functools` module to multiply all these values together
4. Starts with an initial value of 1 (the identity element for multiplication)

The function is defined as:
```python
checksum = lambda s: reduce(lambda x, y: x * y, [ord(c) for c in s], 1)
```

As shown in the test, the checksum of "hello" is 13599570816, which is the product of the ASCII values of 'h' (104), 'e' (101), 'l' (108), 'l' (108), and 'o' (111).

<details>

- id: `msg_01Ks5CRcnZ5yXswHY8E7Dkxa`
- content: `[{'citations': None, 'text': 'I\'ve created a 1-line `checksum` function that:\n1. Takes a string `s` as input\n2. Converts each character to its ASCII value using `ord()`\n3. Uses `reduce()` from the `functools` module to multiply all these values together\n4. Starts with an initial value of 1 (the identity element for multiplication)\n\nThe function is defined as:\n```python\nchecksum = lambda s: reduce(lambda x, y: x * y, [ord(c) for c in s], 1)\n```\n\nAs shown in the test, the checksum of "hello" is 13599570816, which is the product of the ASCII values of \'h\' (104), \'e\' (101), \'l\' (108), \'l\' (108), and \'o\' (111).', 'type': 'text'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 901, 'output_tokens': 196}`

</details>

By asking for a calculation to be done on the username, we force it to use multiple steps:

In [ ]:
pr = 'Use it to get the checksum of the username of this session.'
chat.toolloop(pr, trace_func=_show_cts)

Looking up username
I'll use the `checksum` function to calculate the checksum of your username for this session.

First, I'll get your username:
get_user({})
Press Enter to execute, or enter "n" to skip?
```
username = "Jeremy"
result = checksum(username)
print(f"The checksum of '{username}' is: {result}")
```

Now I'll calculate the checksum of your username using the function we created:
username = "Jeremy"
result = checksum(username)
print(f"The checksum of '{username}' is: {result}")
The checksum of your username "Jeremy" is 1134987783204.

This is calculated by multiplying the ASCII values of each character:
- 'J' (74)
- 'e' (101)
- 'r' (114)
- 'e' (101)
- 'm' (109)
- 'y' (121)

74 × 101 × 114 × 101 × 109 × 121 = 1134987783204


The checksum of your username "Jeremy" is 1134987783204.

This is calculated by multiplying the ASCII values of each character:
- 'J' (74)
- 'e' (101)
- 'r' (114)
- 'e' (101)
- 'm' (109)
- 'y' (121)

74 × 101 × 114 × 101 × 109 × 121 = 1134987783204

<details>

- id: `msg_01JG7RRyaKNC8pu4GzvMvraQ`
- content: `[{'citations': None, 'text': 'The checksum of your username "Jeremy" is 1134987783204.\n\nThis is calculated by multiplying the ASCII values of each character:\n- \'J\' (74)\n- \'e\' (101)\n- \'r\' (114)\n- \'e\' (101)\n- \'m\' (109)\n- \'y\' (121)\n\n74 × 101 × 114 × 101 × 109 × 121 = 1134987783204', 'type': 'text'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1322, 'output_tokens': 118}`

</details>

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()